In [15]:
from random import shuffle
import glob
import csv
import sys
import cv2
import numpy as np
import tensorflow as tf

In [16]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [17]:
def loadImage(addr):
    img = cv2.imread(addr)
    if not img is None:
        # Resize it
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
        # Convert it to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [25]:
def createDataRecord(outputAddr, inputAddrs):
    writer = tf.io.TFRecordWriter(outputAddr)
    
    for i in range(len(inputAddrs)):
        
        if i % 1000 == 0:
            print("{0} images completed".format(i))
        
        imgNeutral = loadImage(inputAddrs[i][0])
        imgExpressive = loadImage(inputAddrs[i][1])
        
        if imgNeutral is None or imgExpressive is None:
            continue
        
        feature = {
            'image_neutral': _bytes_feature(imgNeutral.tostring()),
            'image_expressive': _bytes_feature(imgExpressive.tostring())
        }
        
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [26]:
dbBasePath = 'C:\\Users\\Will\\Documents\\StoicNetData\\'

with open(dbBasePath + r'pairs.txt', newline = '') as pairData:
    pairReader = csv.reader(pairData, delimiter='\t')
    pairs = [[dbBasePath + p for p in pair] for pair in pairReader]

shuffle(pairs)

# 60/20/20 train/val/test split
trainPairs = pairs[0:int(0.6*len(pairs))]
valPairs = pairs[int(0.6*len(pairs)):int(0.8*len(pairs))]
testPairs = pairs[int(0.8*len(pairs)):]


In [27]:
createDataRecord(dbBasePath + 'train.tfrecords', trainPairs)
createDataRecord(dbBasePath + 'val.tfrecords', valPairs)
createDataRecord(dbBasePath + 'test.tfrecords', testPairs)

0 images completed
1000 images completed
2000 images completed
3000 images completed
4000 images completed
5000 images completed
6000 images completed
0 images completed
1000 images completed
2000 images completed
0 images completed
1000 images completed
2000 images completed
